In [124]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import math
import sys

import yfinance as yf
yf.pdr_override()

dataset = pd.read_csv('db/acoes1.csv')
dataset


,Date,VALE3.SA,RECV3.SA,KEPL3.SA,CURY3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,...,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA,BOVA11.SA
0,2021-05-05,110.709999,15.610000,8.475000,9.85,7.810000,24.430000,32.809525,37.627274,26.660000,...,9.307692,8.839826,29.260000,25.250000,43.818180,17.166666,23.830000,67.669998,5.52,115.150002
1,2021-05-06,115.050003,16.000000,8.896666,9.92,8.020000,24.280001,34.304760,38.018181,27.110001,...,9.223076,8.831168,29.209999,26.209999,43.438015,16.933332,23.500000,67.459999,5.52,115.430000
2,2021-05-07,115.449997,16.600000,8.858333,10.09,8.380000,24.469999,34.638096,39.599998,28.020000,...,9.238461,8.952380,29.940001,25.850000,43.214874,17.780001,24.379999,67.419998,5.51,117.459999
3,2021-05-10,114.690002,15.880000,8.895000,10.25,8.170000,23.870001,34.647617,39.900002,28.309999,...,9.115384,9.047619,30.330000,24.500000,44.363636,17.766666,24.700001,68.470001,5.52,117.139999
4,2021-05-11,118.720001,16.150000,8.913333,10.20,8.370000,24.469999,35.857143,40.463634,28.510000,...,9.238461,9.047619,30.549999,23.969999,46.685951,17.616667,25.150000,68.320000,5.57,118.129997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,2023-10-16,67.300003,22.190001,10.760000,15.10,18.860001,40.130001,22.540001,12.960000,23.530001,...,12.230000,9.030000,49.810001,15.530000,37.740002,11.740000,36.680000,57.369999,4.70,112.940002
613,2023-10-17,67.849998,21.660000,10.600000,14.65,18.590000,39.970001,22.330000,12.810000,23.230000,...,12.010000,8.840000,49.360001,15.790000,37.060001,11.270000,37.669998,57.470001,4.75,112.339996
614,2023-10-18,65.360001,21.790001,10.440000,14.29,18.540001,38.900002,21.660000,12.240000,21.990000,...,11.680000,8.750000,49.119999,15.480000,36.310001,11.130000,38.520000,56.080002,4.66,110.370003
615,2023-10-19,64.419998,21.799999,10.420000,14.47,18.580000,38.759998,21.629999,12.340000,21.959999,...,11.780000,8.800000,49.500000,15.470000,36.689999,11.000000,38.340000,55.980000,4.65,110.599998


In [125]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0):
    if seed != 0:
        np.random.seed(seed)

    dataset = dataset.copy()
    
    pesos = np.random.random(len(dataset.columns)-1)
    pesos = pesos/ pesos.sum()
    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['SOMA VALOR'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']

    dataset.drop(columns=['Date'], inplace=True)
    dataset['taxa de retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa de retorno'][i] = ((dataset['SOMA VALOR'][i] / dataset['SOMA VALOR'][i-1]) - 1 )*100

    acoes_pesos = pd.DataFrame(data= {'Ações': colunas, 'Pesos': pesos*100})

    return dataset, datas, acoes_pesos , dataset.loc[len(dataset)-1]['SOMA VALOR']

In [126]:
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, 10)

In [127]:
figura = px.line(x = datas, y = dataset['taxa de retorno'], title=' Retorno diário')
figura.show()

In [128]:
figura = px.line(title=' Evolução patrimonio')

for i in dataset.drop(columns= [ 'SOMA VALOR', 'taxa de retorno']).columns:
    figura.add_scatter(x= datas, y = dataset[i], name = i)
figura.show()    

In [129]:
figura = px.line(x = datas, y = dataset['SOMA VALOR'], title=' Patrimônio')
figura.show()

In [130]:
dataset.loc[len(dataset)-1]['SOMA VALOR']/ dataset.loc[0]['SOMA VALOR'] -1 ## retorno acumulado

0.023368343521429136

In [131]:
dataset['taxa de retorno'].std() ## desvio padrao

1.2668058870076602

In [132]:
# (dataset['taxa de retorno'].mean())/ dataset['taxa de retorno'].std() ## sharp ratio todo o periodo 
(dataset['taxa de retorno'].mean())/ dataset['taxa de retorno'].std() * math.sqrt(246) ## sharp ratio anual 

0.14574448903098003

In [133]:
dinheiro_total = 5000
soma_valor - dinehiro_total

116.84171760714707

In [134]:
taxa_selic_2020 = 12
taxa_selic_2021 = 13.75
taxa_selic_2022 = 13.75
taxa_selic_2023 = 13.25
valor_2020 = dinheiro_total + (dinheiro_total * taxa_selic_2020 / 100)
valor_2021 = valor_2020 + (dinheiro_total * taxa_selic_2021 / 100)
valor_2022 = valor_2021 + (dinheiro_total * taxa_selic_2022 / 100)
valor_2023 = valor_2022 + (dinheiro_total * taxa_selic_2023 / 100)
valor_2023


7637.5

In [135]:
taxa_selic_historico = np.array([taxa_selic_2020, taxa_selic_2021, taxa_selic_2022, taxa_selic_2023])

(dataset['taxa de retorno'].mean() - taxa_selic_historico.mean()/100)/ dataset['taxa de retorno'].std() * math.sqrt(246) #sharp ratio validando historico da selic

-1.4870064915887484

In [136]:
dataset

,VALE3.SA,RECV3.SA,KEPL3.SA,CURY3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,DIRR3.SA,...,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA,BOVA11.SA,SOMA VALOR,taxa de retorno
0,308.185147,8.291549,253.177425,299.188459,199.181051,89.818661,79.137041,303.873974,67.569108,35.296629,...,116.620591,366.701391,285.512445,216.776404,56.804777,149.170229,269.353568,176.536727,5000.000000,0.000000
1,320.266483,8.498705,265.774015,301.314662,204.536765,89.267176,82.743570,307.030901,68.709624,35.628832,...,116.421303,380.643292,283.035348,213.829926,56.018139,148.707312,269.353568,176.965994,5053.631708,1.072634
2,321.379953,8.817406,264.628878,306.478323,213.717957,89.965720,83.547581,319.805497,71.015996,36.265557,...,119.330845,375.415093,281.581398,224.521442,58.115838,148.619135,268.865620,180.078189,5138.775357,1.684801
3,319.264344,8.434964,265.724273,311.338232,208.362256,87.759782,83.570547,322.228290,71.750991,37.732789,...,120.885252,355.809270,289.066551,224.353061,58.878642,150.933738,269.353568,179.587597,5137.512654,-0.024572
4,330.482714,8.578380,266.271927,309.819503,213.462918,89.965720,86.487941,326.780132,72.257888,36.680811,...,121.762096,348.112162,304.198396,222.458903,59.951327,150.603078,271.793374,181.105365,5175.461548,0.738663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,187.344065,11.786641,321.438233,458.654382,480.992930,147.540848,54.366803,104.663621,59.636203,49.553698,...,198.526033,225.539505,245.907980,148.249808,87.435973,126.465141,229.340892,173.148572,5294.336716,0.044876
613,188.875096,11.505121,316.658487,444.985851,474.107017,146.952597,53.860278,103.452240,58.875858,48.418669,...,196.732480,229.315443,241.477204,142.314773,89.795881,126.685584,231.780698,172.228702,5233.513066,-1.148844
614,181.943651,11.574174,311.878713,434.051056,472.831870,143.018665,52.244228,98.848974,55.733109,46.231665,...,195.775914,224.813361,236.590320,140.546883,91.822075,123.621500,227.389056,169.208500,5138.657206,-1.812470
615,179.326951,11.579485,311.281259,439.518468,473.851978,142.503934,52.171866,99.656566,55.657073,46.370081,...,197.290471,224.668143,239.066323,138.905274,91.392999,123.401058,226.901109,169.561107,5143.729209,0.098703


In [137]:
def alocacao_portfolio(dataset, dinheiro_total, sem_risco, repeticoes):
  dataset = dataset.copy()
  dataset_original = dataset.copy()

  lista_retorno_esperado = []
  lista_volatilidade_esperada = []
  lista_sharpe_ratio = []

  melhor_sharpe_ratio = 1 - sys.maxsize
  melhores_pesos = np.empty
  melhor_volatilidade = 0
  melhor_retorno = 0
  
  for _ in range(repeticoes):
    pesos = np.random.random(len(dataset.columns) - 1)
    pesos = pesos / pesos.sum()

    for i in dataset.columns[1:]:
      dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
      dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset.drop(labels = ['Date'], axis = 1, inplace=True)

    retorno_carteira = np.log(dataset / dataset.shift(1))
    matriz_covariancia = retorno_carteira.cov()

    dataset['soma valor'] = dataset.sum(axis = 1)
    dataset['taxa retorno'] = 0.0

    for i in range(1, len(dataset)):
      dataset['taxa retorno'][i] = np.log(dataset['soma valor'][i] / dataset['soma valor'][i - 1])

    #sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)
    retorno_esperado = np.sum(dataset['taxa retorno'].mean() * pesos) * 246
    volatilidade_esperada = np.sqrt(np.dot(pesos, np.dot(matriz_covariancia * 246, pesos)))
    sharpe_ratio = (retorno_esperado - sem_risco) / volatilidade_esperada

    if sharpe_ratio > melhor_sharpe_ratio:
      melhor_sharpe_ratio = sharpe_ratio
      melhores_pesos = pesos
      melhor_volatilidade = volatilidade_esperada
      melhor_retorno = retorno_esperado

    lista_retorno_esperado.append(retorno_esperado)
    lista_volatilidade_esperada.append(volatilidade_esperada)
    lista_sharpe_ratio.append(sharpe_ratio)
    
    dataset = dataset_original.copy()

  return melhor_sharpe_ratio, melhores_pesos, lista_retorno_esperado, lista_volatilidade_esperada, lista_sharpe_ratio, melhor_volatilidade, melhor_retorno

In [138]:
melhor_sharpe_ratio, melhores_pesos = alocacao_portifolio(pd.read_csv('db/acoes1.csv'), 5000, taxa_selic_historico.mean()/100, 100)

KeyError: "['Date'] not found in axis"